In [97]:
from subprocess import check_output
import tempfile
import os

def compare_caseinsensitve(text1,text2):
    return text1.upper().lower() == text2.upper().lower() 

In [98]:
tmp_dir=tempfile.mkdtemp()
mdpfile=os.path.join(tmp_dir,'tmp.mdp')

In [99]:
GMX='/home/micha/software/GROMACS/gromacs_2016.3_ompi-1.10.2_gcc-5.4/inst/oldcpu/bin/gmx'
out = check_output([GMX, "dump", "-s", "topol.tpr", "-om", mdpfile])

In [100]:
out_list = open(mdpfile).read().splitlines()
keywords = dict()
for line in out_list:
    if line.find('=') != -1:
        line_split = line.split("=",1)
        key, value = [l.strip() for l in line_split]
        keywords[key] = value
    else:
        line_split = line.split(":",1)
        key, value = [l.strip() for l in line_split]
        keywords[key] = value

In [101]:
keywords

{'DispCorr': 'No',
 'E-x': '0',
 'E-xt': '0',
 'E-y': '0',
 'E-yt': '0',
 'E-z': '0',
 'E-zt': '0',
 'MMChargeScaleFactor': '1',
 'QMMM': 'false',
 'QMMMscheme': '0',
 'QMconstraints': '0',
 'Shake-SOR': 'false',
 'acc': '0           0           0',
 'annealing': 'No          No',
 'annealing-npoints': '0           0',
 'bd-fric': '0',
 'comm-mode': 'Linear',
 'compressed-x-precision': '100',
 'compressibility': '0.0003 0.0003 0.0003 0 0 0',
 'constraint-algorithm': 'Lincs',
 'continuation': 'false',
 'cos-acceleration': '0',
 'coulomb-modifier': 'Potential-shift',
 'coulombtype': 'Reaction-Field',
 'cutoff-scheme': 'Verlet',
 'deform': '0 0 0 0 0 0',
 'disre': 'No',
 'disre-mixed': 'false',
 'disre-weighting': 'Conservative',
 'dr-fc': '1000',
 'dr-tau': '0',
 'dt': '0.01',
 'emstep': '0.01',
 'emtol': '10',
 'energygrp-flags[  0]': '0 0',
 'energygrp-flags[  1]': '0 0',
 'epsilon-r': '15',
 'epsilon-rf': 'inf',
 'epsilon-surface': '0',
 'ewald-geometry': '0',
 'ewald-rtol': '1e-05',


In [117]:
def map_gromacs_to_database(keywords):
    
    
    rv = dict()
    
    # mapping general stuff
    if 'dt' in keywords.keys():
        rv['dt'] = float(keywords['dt'])
        rv['nsteps'] = int(keywords['nsteps'])
    
    # mapping barostats
    if 'pcoupl' in keywords.keys():
        if compare_caseinsensitve(keywords['pcoupl'],'Berendsen'):
            rv['barostat_type'] = 'Berendsen'
        elif compare_caseinsensitve(keywords['pcoupl'],'Parrinello-Rahman'):
            rv['barostat_type'] = 'Parrinello-Rahman'
        elif compare_caseinsensitve(keywords['pcoupl'],'MTTK'):
            rv['barostat_type'] = 'Parrinello-Rahman'
            rv['p_MKT'] = True
            
        if keywords['pcoupl'] != 'no':
            # barostat coupling
            if compare_caseinsensitve(keywords['pcoupltype'],'isotropic'):
                rv['p_coupling'] = 'iso'
                n_p = 1
            elif compare_caseinsensitve(keywords['pcoupltype'],'semiisotropic'):
                rv['p_coupling'] = 'xy'
                n_p = 2
            elif compare_caseinsensitve(keywords['pcoupltype'],'anisotropic'): 
                rv['p_coupling'] = 'aniso'
                n_p = 6
            elif compare_caseinsensitve(keywords['pcoupltype'],'surface-tension'):
                rv['p_coupling'] = 'surface-tension'
                n_p = 33
            
            # general informations about the batostat
            rv['p_relax'] = keywords['tau-p']
            rv['p_target'] = " ".join(keywords['ref-p'].split()[:n_p])
            rv['p_compressibility'] = " ".join(keywords['compressibility'].split()[:n_p])
    
    # mapping thermostat
    if 'tcoupl' in keywords.keys():
        if compare_caseinsensitve(keywords['tcoupl'],'berendsen'):
            rv['thermostat_type'] = 'Berendsen'
        elif compare_caseinsensitve(keywords['tcoupl'],'nose-hoover'):
            rv['thermostat_type'] = 'Nose-Hoover'
            rv['T_nh_chain'] = keywords['nh-chain-length']
        elif compare_caseinsensitve(keywords['tcoupl'],'andersen'):
            rv['thermostat_type'] = 'Andersen'
        elif compare_caseinsensitve(keywords['tcoupl'],'andersen-massive'):
            rv['thermostat_type'] = 'Andersen-massive'
        elif compare_caseinsensitve(keywords['tcoupl'],'v-rescale'):
            rv['thermostat_type'] = 'v-rescale'
            
        if keywords['tcoupl'] != 'no':
            T_relax = list(set(keywords['tau-t'].split()))
            assert len(T_relax) == 1, 'Not implemented different groups'
            rv['T_relax'] = T_relax[0]
            T_target = list(set(keywords['ref-t'].split()))
            assert len(T_relax) == 1, 'Not implemented different groups'
            rv['T_target'] = T_target[0]
            
    return rv

In [118]:
map_gromacs_to_database(keywords)

{'T_relax': '1',
 'T_target': '300',
 'barostat_type': 'Parrinello-Rahman',
 'dt': 0.01,
 'nsteps': 1000000000,
 'p_compressibility': '0.0003',
 'p_coupling': 'iso',
 'p_relax': '12',
 'p_target': '1',
 'thermostat_type': 'v-rescale'}

True